# Carregando Pacotes

In [1]:
from transformers import pipeline
import torch
import pdfplumber
from transformers import AutoTokenizer

from sentence_transformers import SentenceTransformer

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:


# Verifica e configura o dispositivo (GPU/CPU)
device = 0 if torch.cuda.is_available() else -1  # 0 = GPU, -1 = CPU
print(f"🔧 Rodando em: {'GPU' if device == 0 else 'CPU'}")

🔧 Rodando em: GPU


# Variáveis Globais

In [3]:
#Tamanho da Janela de Contexto
window_size=400

#Overlap da Janela de Contexto
stride=100

#Tamanho da Chunk
chunk_size = 400
    
#Overlap da Chunk
overlap = 100

#Tamanho Máximo da Pergunta
tamanho_pergunta = 60

In [4]:
modelo_tokenizador = "pierreguillou/bert-large-cased-squad-v1.1-portuguese"

# Tratamento do Documento

In [12]:
# Carregar modelo BERTimbau pré-treinado para Q&A em Portugues
tokenizer = AutoTokenizer.from_pretrained(modelo_tokenizador)

qa_pipeline = pipeline(
    "question-answering",
    model= modelo_tokenizador,  # Modelo em Portugues
    tokenizer=modelo_tokenizador,
    device=device
)

Device set to use cuda:0


In [5]:
def carregar_documento(caminho_do_pdf):

    with pdfplumber.open(caminho_do_pdf) as pdf:
        texto = " ".join(
            page.extract_text() for page in pdf.pages 
            if page.extract_text()  # Ignora páginas sem texto
        )
        texto = " ".join(texto.split())  # Normaliza espaços
    return texto

## Divisao em Chunks

In [6]:
def dividir_em_chunks_tokenizados(texto):

    tokens = tokenizer.tokenize(texto)
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        chunks.append(tokenizer.convert_tokens_to_string(chunk))
    return chunks

## Janela Deslizante

In [13]:
def sliding_window_tokenizados(texto):
    tokens = tokenizer.tokenize(texto)
    janelas = []
    
    for i in range(0, len(tokens), stride):
        janela = tokens[i:i + window_size]
        texto_janela = tokenizer.convert_tokens_to_string(janela)
        janelas.append(texto_janela)
        
        # Para se chegarmos ao final do texto
        if i + window_size >= len(tokens):
            break
    
    return janelas

## Selecionar Divisão do Texto

In [14]:
#Divisao em Janela Deslizante

divisao_do_texto = sliding_window_tokenizados(documento)

#Divisao em Chunks

#divisao_do_texto = dividir_em_chunks_tokenizados(documento)

In [9]:
documento = carregar_documento('Cabelo Cacheado Maior.pdf')

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


# Tokenização dos Chunks

In [15]:
modelo_vetorizador = SentenceTransformer('all-MiniLM-L6-v2')

# Vetorizar todos os chunks
vetores = modelo_vetorizador.encode(divisao_do_texto, convert_to_tensor=True)


# Função Recuperação

In [16]:
def recuperar_chunks(pergunta, modelo_vetorizador, divisao_do_texto, k=3):
    # Vetoriza a pergunta
    vetor_pergunta = modelo_vetorizador.encode([pergunta], convert_to_tensor=False)
    
    # Vetoriza todos os chunks
    vetores_chunks = modelo_vetorizador.encode(divisao_do_texto, convert_to_tensor=False)
    
    # Calcula similaridade
    sim = cosine_similarity(vetor_pergunta, vetores_chunks)
    
    # Pega índices dos top-k chunks mais similares
    top_k_indices = np.argsort(sim[0])[::-1][:k]
    
    chunks_relevantes = [divisao_do_texto[i] for i in top_k_indices]
    
    return chunks_relevantes


# Modelo Gerador

In [ ]:
modelo_gerador = pipeline("text2text-generation", model="facebook/bart-large", device=device)

def gerar_resposta(pergunta, chunks_relevantes):
    contexto = " ".join(chunks_relevantes)
    prompt = f"Contexto: {contexto} \n\nPergunta: {pergunta} \n\nResposta:"
    resposta = modelo_gerador(prompt, max_length=200)
    return resposta[0]['generated_text']


config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

# Processamento

In [21]:
pergunta = "Quais são os tipos de cachos?"

chunks_relevantes = recuperar_chunks(pergunta, modelo_vetorizador, divisao_do_texto)

resposta = gerar_resposta(pergunta, chunks_relevantes)

print("📝 Resposta:", resposta)

Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


📝 Resposta: Os cabelos do Tipo 2, por exemplo, so considerados ondulados, transitando entre o liso e o cacheado
